In [1]:
import numpy as np
import matplotlib.pyplot as plt
import timewarp_lib.load_model as lm
import timewarp_lib.train_utils as tu
import torch
import pandas as pd
import json
import numpy as np
import matplotlib

/home/travers/miniconda3/envs/timewarpvae/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
model = lm.LoadedModel("../results/rescaled/20231011-003229.166126/savedmodel")

In [31]:
#model = lm.LoadedModel("../results/retrainedforkdata/20230928-065206.641930/savedmodel")

In [32]:
cmap = plt.get_cmap("viridis")

In [33]:
fork_modeldata_info = np.load("../forkdata/forkTrajectoryData.npz")

In [34]:
fork_test_data = fork_modeldata_info["test"]
fork_test_data.shape

(105, 200, 7)

In [35]:

####clean_model_dirs = ["../results/overnight/20230921-093159.705744/savedmodel"] # TenDTWModel



In [36]:
import matplotlib
font = {        'size'   : 22}

matplotlib.rc('font', **font)

In [37]:
datatype = "train"
dtype=torch.float
device="cpu"
batch_size=100
training_data_timing_noise=0

dataName = "../data/trainTest2DLetterARescaled.npz"
# the datafile has information on how the data was cleaned
# the learned/applied model is on clean data
# so we need to convert back to trajectory data
loaded_data_dict = np.load(dataName)

ydata = torch.tensor(loaded_data_dict[datatype],dtype=dtype).to(device)
np_ydata = ydata.detach().cpu().numpy()
num_trajs, numts, traj_channels = ydata.shape
tdata = torch.tensor(np.linspace(0,1,numts),dtype=dtype).to(device).expand(num_trajs,numts).unsqueeze(2)

torch_train_data = torch.utils.data.TensorDataset(tdata, ydata)
training_dataloader = torch.utils.data.DataLoader(torch_train_data, batch_size=batch_size, shuffle=True)

In [38]:
ydata.shape

torch.Size([125, 200, 2])

In [39]:
model.model.scalar_timewarper

ModeledParameterScalarTimewarper(
  (timewarp_parameter_encoder): OneDConvEncoder(
    (fcmu): Linear(in_features=1408, out_features=50, bias=True)
    (fclogvar): Linear(in_features=1408, out_features=50, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (nonlinearity): ReLU()
    (emb_convs): ModuleList(
      (0): Conv1d(2, 16, kernel_size=(3,), stride=(1,))
      (1): Conv1d(16, 32, kernel_size=(3,), stride=(2,))
      (2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
      (3): Conv1d(32, 64, kernel_size=(3,), stride=(2,))
      (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
      (5): Conv1d(64, 64, kernel_size=(3,), stride=(2,))
    )
    (emb_fcs): ModuleList()
  )
  (LogSoftmaxLayer): LogSoftmax(dim=1)
  (monotonic_applier): ParameterizedMonotonicApplier()
)

In [40]:
model.model.decoder

ComplicatedFunctionStyleDecoder(
  (motion_model): TemplateMotionGeneration(
    (nonlinearity): ELU(alpha=1.0)
    (all_layers): ModuleList(
      (0): Linear(in_features=1, out_features=500, bias=True)
      (1): Linear(in_features=500, out_features=500, bias=True)
      (2): Linear(in_features=500, out_features=64, bias=True)
    )
  )
  (nonlinearity): ELU(alpha=1.0)
  (all_side_layers): ModuleList(
    (0): Linear(in_features=16, out_features=200, bias=True)
    (1): Linear(in_features=200, out_features=128, bias=True)
  )
)

In [41]:
emb = model.model.encoder.encode(ydata[(0,1),],tdata[(0,1),])[0]

before transpose torch.Size([2, 200, 2])
after transpose torch.Size([2, 2, 200])
did convolution  torch.Size([2, 16, 198])
did convolution  torch.Size([2, 32, 98])
did convolution  torch.Size([2, 64, 48])
did convolution  torch.Size([2, 32, 23])
flattened torch.Size([2, 736])
final torch.Size([2, 16])


In [42]:
model.model.scalar_timewarper.timewarp_parameter_encoder.encode(ydata[(0,1),],tdata[(0,1),])[0]

before transpose torch.Size([2, 200, 2])
after transpose torch.Size([2, 2, 200])
did convolution  torch.Size([2, 16, 198])
did convolution  torch.Size([2, 32, 98])
did convolution  torch.Size([2, 32, 96])
did convolution  torch.Size([2, 64, 47])
did convolution  torch.Size([2, 64, 45])
did convolution  torch.Size([2, 64, 22])
flattened torch.Size([2, 1408])
final torch.Size([2, 50])


tensor([[-0.9532, -0.2034,  0.3342,  0.4481,  0.3413,  0.5072,  0.5424,  0.5480,
          0.3798,  0.1958,  0.0779, -0.0119,  0.1796,  0.2761,  0.4575,  0.5816,
          0.5991,  0.6518,  0.4834,  0.4685,  0.4266,  0.4544,  0.5001,  0.5344,
          0.6760,  0.7994,  0.8417,  0.8305,  0.8506,  0.8199,  0.8189,  0.6181,
          0.6393,  0.6508,  0.7593,  0.7562,  0.6575,  0.4780,  0.3167,  0.0570,
          0.0675,  0.0561, -0.1871,  0.0088, -0.1459,  0.0286,  0.0778, -0.2034,
          0.1688, -0.2595],
        [-0.7832, -0.1565,  0.0728,  0.0595, -0.1389, -0.0803,  0.3789,  0.5212,
          0.6676,  0.6181,  0.7337,  0.2842,  0.5973,  0.4844,  0.5811,  0.6310,
          0.5097,  0.7347,  0.5804,  0.6681,  0.6587,  0.6985,  0.7594,  0.6966,
          0.8075,  0.8348,  0.8739,  0.8189,  0.9008,  0.7661,  0.9679,  0.5854,
          0.6219,  0.3453,  0.6477,  0.6709,  0.6299,  0.6548,  0.3300,  0.4896,
          0.3027,  0.4301,  0.3639,  0.2435,  0.2734, -0.0101, -0.2588, -0.4587,


In [43]:
answer = model.model.decoder.decode(emb[(0,1),],tdata[(0,1),]);

motion_model_input torch.Size([400, 1])
after fc:  torch.Size([400, 500])
after fc:  torch.Size([400, 500])
after fc:  torch.Size([400, 64])
